In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install flask-ngrok

In [ ]:
import flask
import werkzeug
import keras.models
import numpy
import scipy.misc
import imageio
import cv2
from flask_ngrok import run_with_ngrok
import numpy as np

app = flask.Flask(__name__)
run_with_ngrok(app)


@app.route('/', methods=['GET', 'POST'])
def handle_request():
    imagefile = flask.request.files['image']
    filename = werkzeug.utils.secure_filename(imagefile.filename)
    print("\nReceived image File name : " + imagefile.filename)
    imagefile.save(filename)
    emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}

    frame = cv2.imread(filename)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Aura/haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
      cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
      roi_gray = gray[y:y + h, x:x + w]
      cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
      cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
      loaded_model = keras.models.load_model('/content/drive/MyDrive/Aura/models/model.h5')
      predicted_label = loaded_model.predict(cropped_img)
      res = emotion_dict[int(np.argmax(predicted_label))]
      print(res)

      return str(res)


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d4e2498bcf58.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040

Received image File name : androidFlask.jpg


127.0.0.1 - - [07/Jun/2021 07:35:16] "POST / HTTP/1.1" 200 -


Happy
